# Preprocessing
Tobias C. Haase \
Master's student of Psychology at [Goethe-University Frankfurt](https://www.goethe-university-frankfurt.de/en?locale=en)


Dataset: [Abrupt hippocampal remapping signals resolution of memory interference](https://openneuro.org/datasets/ds003707/versions/1.0.0)

## Import of the raw Data
Using datalad, the data of the first person is imported below. 

In [ ]:
#! cd /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/
#!datalad install https://github.com/OpenNeuroDatasets/ds003707.git
#! cd ./ds003707
!datalad get sub-01
#!datalad get sub-*

## Quality Controle with MRI-QC
Using docker, I will first compute quality measures for each participant. 
More information is available [here](https://mriqc.readthedocs.io/en/latest/docker.html)

This will first be done for participant one, to ensure that there is no mistake. Full data is also imported later!

The following line is work in progress. Perhaps still need to swap pipepline from nipreps. 

In [ ]:
!docker run -it --rm -v  /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/ds003707:/data:ro \
        -v /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/derivatives:/out nipreps/mriqc:16.0.0 \
        /data /out participant --participant_label 01

## Preprocessing via fMRI-Prep

Multiple attempts to preprocess failed. In the following they are outlined. 

### Attempet 1

In [ ]:
!docker run -it --rm -v  /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/ds003707:/data:ro \
-v /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/derivatives:/out -v  /home/tchaase/licenses/freesurfer/license.txt:/opt/freesurfer/license.txt nipreps/fmriprep:21.0.4  \
/data /out participant --participant_label 01 --error-on-aroma-warnings --ignore t2w --fs-license-file /opt/freesurfer/license.txt

This produced an error related to the `resample` workflow. The respective error-message (bash output and error-log) is in the folder `attempt_1`.  

Following this attempt, I specified `output-spaced`. Also the correct argument regarding aroma (`--use-armoa`) was inserted. 


### Attempet 2

In [ ]:
!docker run -it --rm -v  /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/ds003707:/data:ro \
    -v /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/derivatives:/out -v  /home/tchaase/licenses/freesurfer/license.txt:/opt/freesurfer/license.txt nipreps/fmriprep:21.0.4  \
    /data /out participant --participant_label 01 --use-aroma  --ignore t2w --output-spaces MNI152NLin6Asym:res-2 T1w --fs-license-file /opt/freesurfer/license.txt

This still produced errors, it was suspected it might be due to memory limitations. The respective error-message is in: `fmriprep_error_2`.
After deleting contents of the `work-dir`, the following was attempted.

### Attempt 3

In [ ]:
!docker run -it --rm -m 10GB --cpus="3" -v /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/ds003707:/data:ro \
-v /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/derivatives:/out -v  /home/tchaase/licenses/freesurfer/license.txt:/opt/freesurfer/license.txt nipreps/fmriprep:21.0.4  \
/data /out participant --participant_label 01 --use-aroma  --ignore t2w --output-spaces MNI152NLin6Asym:res-2 T1w --fs-license-file /opt/freesurfer/license.txt 

This lead to early abruption of the process, abrupton of the process. The respective error message is in:`fmriprep_error_3`.

This error persists unless the `-m`argument is removed. As an attempt to solve this issue, the working directory was specified as shown below:

### Attempt 3

In [ ]:
!docker run -it --rm -m 10gb --cpus="3" -v  /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/ds003707:/data:ro \
    -v /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/derivatives:/out -v /home/tchaase/Documents/Universitaet/Forschungsmodul/project/data/derivatives/workdir:/workdir  \
    -v  /home/tchaase/licenses/freesurfer/license.txt:/opt/freesurfer/license.txt nipreps/fmriprep:21.0.4  \
    /data /out participant --participant_label 01 --use-aroma  --ignore t2w --output-spaces MNI152NLin6Asym:res-2 T1w -w /workdir

The respective error message is in `fmriprep_error_4`. Attempt 5 contains the same argument above without the `-m 10gb --cpus="3"` arguments. 
